In [64]:
## coding: utf-8
# 3msの信号を読み込む
# mfかけて１波ぶん抽出
# テキストファイルに出力

import wave
import scipy as sp
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from scipy.fftpack import fft
import scipy.signal as sg
from scipy import signal
import pandas as pd
import scipy.interpolate as itpl
import resampy

def readwav2(file1):
    wr = wave.open(file1, 'r')
    # waveファイルが持つ性質を取得
    ch = wr.getnchannels()
    width = wr.getsampwidth()
    fr = wr.getframerate()
    fn = wr.getnframes()
    data = wr.readframes(wr.getnframes())
    wr.close()
    X = np.frombuffer(data, dtype=np.int16)
    # 波全体の時間幅
    t = np.arange(0,len(X))/int(fr)
    # 1秒間の時間幅
    t_1sec = np.arange(0,int(fr))/int(fr)
    # 時間刻み幅
    dt = 1/int(fr)
    # 開始時間の指標
    start = int(2.0/dt)
    # 終了時間の指標
    end = int(5.0/dt)
    # 3秒のみ切り出した波形
    X_3sec = X[start:end:1]
    # plt.plot(t_1sec,X_1sec)
    return X

def chirp_exp(numSamples, chirpLen_s, start_Hz, stop_Hz, phase_rad):
    # チャープの生成関数,chirpaは生成したチャープ信号の配列を返す
    times_s = np.linspace(0, chirpLen_s, numSamples) # Chirp times.
    k = (stop_Hz - start_Hz) / chirpLen_s # Chirp rate.
    sweepFreqs_Hz = (start_Hz + k/2. * times_s) * times_s
    chirpa = np.array(np.exp(phase_rad*1j-2*np.pi*1j*sweepFreqs_Hz))
    return chirpa

def mf_output(file,chirp):
    mf_out = sg.correlate(file[48000:48000*3],chirp, mode='same', method='auto')
    #最大値のインデックス見つける
    index = mf_out.argmax()
    #1つ分の波の抽出
    mf_1sec = file[48000+index-72:48000+index+72]
    return mf_1sec

if __name__ == '__main__':
    
    speaker_deg = 170
    mic_deg = -25
    output = []
    hz =10
    
    for i in range(0,7):
        # 測位データ名
        filename = 'test_new/2022-02-28/s'+str(speaker_deg)+'m'+str(mic_deg)+'_'+str(hz)+'k'+str(hz+1)+'k.wav'
        XX = readwav2(filename)
        C = chirp_exp(144,0.003,hz*1000,(hz+1)*1000,np.pi*0.5)
        output.extend(mf_output(XX,C) )
        #print(output)
        hz += 1
    outputnp = np.array(output)
    print(outputnp)
    print(len(outputnp))
    np.savetxt('s'+str(speaker_deg)+'m'+str(mic_deg)+'new.txt',outputnp)
    '''
    for speaker in range (0,7):
        mic_deg = -40
        for mic in range(0,9):
            output = []
            hz =10
            for i in range(0,7):
                # 測位データ名
                filename = 'speaker'+str(speaker_deg)+'/2022-02-28/s'+str(speaker_deg)+'m'+str(mic_deg)+'_'+str(hz)+'k'+str(hz+1)+'k.wav'
                XX = readwav2(filename)
                C = chirp_exp(144,0.003,hz*1000,(hz+1)*1000,np.pi*0.5)
                output.extend(mf_output(XX,C) )
                #print(output)
                hz += 1
            outputnp = np.array(output)
            print(outputnp)
            print(len(outputnp))
            np.savetxt('s'+str(speaker_deg)+'m'+str(mic_deg)+'.txt',outputnp)
            mic_deg += 10
        speaker_deg += 30
    '''

[ 380 1374  214 ...  725  489  566]
1008
